In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chat-slang-abbreviations-acronyms/slang/slang.pkl
/kaggle/input/chat-slang-abbreviations-acronyms/slang/slang.csv
/kaggle/input/chat-slang-abbreviations-acronyms/slang/slang.txt
/kaggle/input/chat-slang-abbreviations-acronyms/slang/slang.json
/kaggle/input/customer-support-on-twitter/sample.csv
/kaggle/input/customer-support-on-twitter/twcs/twcs.csv


In [2]:
df = pd.read_csv("/kaggle/input/customer-support-on-twitter/twcs/twcs.csv")

In [3]:
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


### Convert the tweets to lower case 

In [4]:
df['text']= df['text'].apply(lambda x: x.lower())

### Punctuation Removal

In [5]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [6]:
def remove_punctuation (text): 
    punctuationfree = "".join([i for i in text if i not in string.punctuation])
    return punctuationfree 

In [7]:
## Applying the Punctuation free function for the 
df['text_pc'] =  df['text'].apply(lambda x: remove_punctuation(x))

### Function for Abbreviations 

In [8]:
## Read the Abbreviations from the Text and Convert it back into the 

def read_text_file(filename):
    data = {}  # Create an empty dictionary to store key-value pairs
    with open(filename, 'r') as file:
        for line in file:
            # Split each line by '=' to separate key and value
            key, value = line.strip().split('	')
            # Store key-value pair in the dictionary
            data[key] = value
    return data

filename = '/kaggle/input/chat-slang-abbreviations-acronyms/slang/slang.txt'
data_dict = read_text_file(filename)

In [9]:
## Function for replacing the Slang words with full Text 

def chat_conversion(text):
    new_text=[]
    for w in text.split():
        if w.upper() in data_dict:
            new_text.append(data_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [10]:
## Applying the Punctuation free function for the 
df['text_pc_sl'] =  df['text_pc'].apply(lambda x: chat_conversion(x))

### Stop words Removal 

In [11]:
import nltk
#Stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')

In [12]:
def remove_stopwords(text):
    new_text = []
    for word in text.split():
            if word in stopwords:
                new_text.append("")
            else :
                new_text.append(word)     
    return(" ".join(new_text))

In [13]:
## Sanity check on the data 
t = ' We are learning this that i am NLP course'
print(t)
print("after the funciton")
remove_stopwords(t)

 We are learning this that i am NLP course
after the funciton


'We  learning     NLP course'

In [ ]:
## Applying the Punctuation free function for the 
df['text_pc_sl_sw'] =  df['text_pc_sl'].apply(lambda x: remove_stopwords(x))

In [ ]:
df[['text_pc_sl_sw','text_pc_sl']].head(5)

In [27]:
df.head(10)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id,text_pc,text_pc_sl,text_pc_sl_sw,text_pc_sl_sw_em
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 i understand. i would like to assist y...,2,3.0,115712 i understand i would like to assist you...,115712 i understand i would like to assist you...,115712 understand would like assist would...,115712 understand would like assist would...
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0,sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare i have sent several private messag...,1,4.0,sprintcare i have sent several private message...,sprintcare i have sent several private message...,sprintcare sent several private messages o...,sprintcare sent several private messages o...
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 please send us a private message so th...,3,5.0,115712 please send us a private message so tha...,115712 please send us a private message so tha...,115712 please send us private message as...,115712 please send us private message as...
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare i did.,4,6.0,sprintcare i did,sprintcare i did,sprintcare,sprintcare
5,6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,@115712 can you please send us a private messa...,"5,7",8.0,115712 can you please send us a private messag...,115712 can you please send us a private messag...,115712 please send us private message g...,115712 please send us private message g...
6,8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",NaN,sprintcare is the worst customer service,sprintcare is the worst customer service,sprintcare worst customer service,sprintcare worst customer service
7,11,sprintcare,False,Tue Oct 31 22:10:35 +0000 2017,@115713 this is saddening to hear. please shoo...,NaN,12.0,115713 this is saddening to hear please shoot ...,115713 this is saddening to hear please shoot ...,115713 saddening hear please shoot us dm ...,115713 saddening hear please shoot us dm ...
8,12,115713,True,Tue Oct 31 22:04:47 +0000 2017,@sprintcare you gonna magically change your co...,"11,13,14",15.0,sprintcare you gonna magically change your con...,sprintcare you gonna magically change your con...,sprintcare gonna magically change connectivi...,sprintcare gonna magically change connectivi...
9,15,sprintcare,False,Tue Oct 31 20:03:31 +0000 2017,@115713 we understand your concerns and we'd l...,12,16.0,115713 we understand your concerns and wed lik...,115713 we understand your concerns and wed lik...,115713 understand concerns wed like plea...,115713 understand concerns wed like plea...


### Function to convert the Emoji's into Text 

In [23]:
pip install demoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [24]:
import demoji
demoji.download_codes()

/tmp/ipykernel_33/2319952594.py:2: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
import emoji

In [25]:
def remove_emoj(text):
    clean_text = demoji.findall(text)
    return(clean_text)

In [26]:
## Applying the Punctuation free function for the 
df['text_pc_sl_sw_em'] =  df['text_pc_sl_sw'].apply(lambda x: remove_emoj(x))


KeyboardInterrupt



### Lemmatization 

In [28]:
from nltk.stem import PorterStemmer

In [29]:
def stemmings(text):
    obj = PorterStemmer()
    
    stem_words = [obj.stem(word) for word in text.split()]
    return stem_words

In [30]:
#applying function to the column
df['text_token_stem']= df['text_pc_sl_sw_em'].apply(lambda x: stemmings(x))

### Tokenization 

In [ ]:
import nltk 
from nltk import word_tokenize,sent_tokenize

In [ ]:
#applying function to the column
df['text_token']= df['text_token_stem'].apply(lambda x: word_tokenize(x))